In [ ]:
import requests
import json
import pandas as pd

## Get FIPS List and Filter down to Tennessee Counties

In [ ]:
fips = pd.read_html('https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697')[0]

In [ ]:
fips = fips[:-1]

In [ ]:
fips.to_csv('data/FIPS.csv', index = False)

In [ ]:
TN_FIPS = list(fips.query('State == "TN"').FIPS)

In [ ]:
TN_Counties = ['LAUCN' + x +'0000000003' for x in TN_FIPS]

## Use the BLS API to get Unemployment Data by County

In [ ]:
unemployment = pd.DataFrame()
for i in range(4):
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": TN_Counties[25*i:25*(i+1)],"startyear":"2018", "endyear":"2018"})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    
    for j,FIPS in enumerate(TN_FIPS[25*i:25*(i+1)]):
        temp = pd.DataFrame.from_records(json_data['Results']['series'][j]['data'])
        temp = temp.drop(columns = ['footnotes'])
        temp['FIPS'] = FIPS
        unemployment = unemployment.append(temp)

In [ ]:
unemployment.to_csv('../../data/raw/unemployment_2018.csv', index = False)